In [0]:
# RUN only once!! hacked way of getting data without mounting google drive and so on... tedious to mount and get auth code....
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1iUQc0KZ0Dit2iglPyEOF9q1dfNc4ERRp' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1iUQc0KZ0Dit2iglPyEOF9q1dfNc4ERRp" -O haha.zip && rm -rf /tmp/cookies.txt
!mkdir data
!unzip haha.zip

--2020-05-27 12:56:54--  https://docs.google.com/uc?export=download&confirm=Etvm&id=1iUQc0KZ0Dit2iglPyEOF9q1dfNc4ERRp
Resolving docs.google.com (docs.google.com)... 108.177.12.139, 108.177.12.102, 108.177.12.101, ...
Connecting to docs.google.com (docs.google.com)|108.177.12.139|:443... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://doc-0o-44-docs.googleusercontent.com/docs/securesc/r9fcljn8dro3gh21hifcupke6qmtdb7r/69plhqfv6876d7fpjq5hn9c4ca4bh8nh/1590584175000/06685749744218960586/08955651176254237751Z/1iUQc0KZ0Dit2iglPyEOF9q1dfNc4ERRp?e=download [following]
--2020-05-27 12:56:54--  https://doc-0o-44-docs.googleusercontent.com/docs/securesc/r9fcljn8dro3gh21hifcupke6qmtdb7r/69plhqfv6876d7fpjq5hn9c4ca4bh8nh/1590584175000/06685749744218960586/08955651176254237751Z/1iUQc0KZ0Dit2iglPyEOF9q1dfNc4ERRp?e=download
Resolving doc-0o-44-docs.googleusercontent.com (doc-0o-44-docs.googleusercontent.com)... 173.194.210.132, 2607:f8b0:400c:c0f::84
Connectin

In [0]:
import os
import gc
import time
import math
import datetime
import tensorflow as tf
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")

In [0]:
def get_num_type(max_value, int_flag):
    if int_flag:
        #if max_value <= np.iinfo(np.uint8).max:
        #    return np.int8 fucks avg up?!!!!
        if max_value <= np.iinfo(np.uint16).max:
            return np.int16
        elif max_value <= np.iinfo(np.uint32).max:
           return np.int32
        elif max_value <=np.iinfo(np.uint64).max:
            return np.int64
        else:
            return np.float32 #everything float32

#assumes no negative values in dataframe!
#only reduces memory usage of numeric values!
def reduce_mem_usage(df, verbose=True):
    numerics = ['int8', 'int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        #print(col)
        col_type = df[col].dtypes
        if col_type in numerics:
            #min_value = df[col].min()
            max_value = df[col].max()
            df[col]= df[col].astype(get_num_type(max_value, "int" in str(col_type)))
        elif col == "date":
            print("handling date")
            df[col] = pd.to_datetime(df[col])
        elif col == "d":# or col=="id":
            pass #do nothing 
        elif col_type == np.object:
            df[col] = df[col].astype("category")
            df[col] = df[col].cat.codes.astype("int16")
            df[col] -= df[col].min()
        #else:
        #    print(col_type)
        #    raise ValueError("Unsupported column type")
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

In [0]:
INPUT_DIR = 'data'
calendar = pd.read_csv(f'{INPUT_DIR}/calendar.csv')
selling_prices = pd.read_csv(f'{INPUT_DIR}/sell_prices.csv')
sales_train_val = pd.read_csv(f'{INPUT_DIR}/sales_train_validation.csv')
#calendar = reduce_mem_usage(pd.read_csv(f'{INPUT_DIR}/calendar.csv'))
#selling_prices = reduce_mem_usage(pd.read_csv(f'{INPUT_DIR}/sell_prices.csv'))
#sample_submission = pd.read_csv(f'{INPUT_DIR}/sample_submission.csv')
#sales_train_val = reduce_mem_usage(pd.read_csv(f'{INPUT_DIR}/sales_train_validation.csv'))

In [0]:
lastday = 1913
num_forecasts = 28

In [0]:
traincols = [f"d_{day}" for day in range(1,lastday+1-num_forecasts)]
targetcols = [f"d_{day}" for day in range(lastday-num_forecasts+1,lastday+1)]
sales = sales_train_val[traincols]#.astype("int32")
last_28 = sales_train_val[targetcols]#.astype("int32")

In [0]:
def get_avgs(data, window, offset):
    print(window, offset, offset-window)
    print(data.iloc[:,offset-window:].shape)
    avg = data.iloc[:,offset-window:].rolling(window,axis=1).mean().astype("float32")
    avg.dropna(axis=1,inplace=True)
    print(avg.shape)
    return avg

In [0]:
def get_sell_prices1(cal, sellp):
    #sellp = filtered[["wm_yr_wk","sell_price"]]
    res = pd.merge(sellp, cal, how="outer", on="wm_yr_wk", sort=True)
    res.fillna(method="backfill", inplace=True)
    return res
def get_sell_prices_mat(sellp):
    temp = selling_prices.groupby(["store_id", "item_id"], sort=False)
    print("Grouped")
    temp = temp.apply(lambda x: get_sell_prices1(cal_wm, x))
    print("applying transformation!")
    temp.reset_index(inplace=True, drop=True)
    print("Drop multikey")
    return temp
    #sellp.groupby(["store_id", "item_id"], sort=False).apply(lambda x: get_sell_prices1(cal_wm, sellp))["sell_price"].values.reshape(30490,1969)
    #return temp["sell_price"].values.reshape(30490,1969)

In [0]:
avg_month = get_avgs(sales, 28, 28)
avg_week = get_avgs(sales, 7, 28)
#sales = sales.iloc[:,27:].values

28 28 0
(30490, 1885)
(30490, 1858)
7 28 21
(30490, 1864)
(30490, 1858)


In [0]:
#sales.rolling(7,axis=1).mean().astype("float32")
#sales_train_val.loc[:,"d_1":"d_1885"]

In [0]:
%%time
cal_wm = calendar["wm_yr_wk"]
sell_prices=get_sell_prices_mat(selling_prices)

In [0]:
sell_prices = sell_prices["sell_price"].values.reshape(30490,1969)

In [0]:
print(sales.shape)
print(avg_month.shape)
print(avg_week.shape)
print(sell_prices.shape)

(30490, 1885)
(30490, 1831)
(30490, 1852)
(30490, 1969)


In [0]:
data = np.dstack([sales, avg_week,avg_month])

In [0]:
data.shape

In [0]:
last_28.shape

In [0]:
from tensorflow.keras.layers import Conv1D, Dense, MaxPooling1D, Flatten
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import numpy as np
inputs = tf.keras.Input(shape=(data.shape[1],data.shape[2]))
X_train, X_test, y_train, y_test = train_test_split(data, last_28)
inputs


In [0]:
X_test[0].shape

In [1]:
conv_month = Conv1D(filters=32, kernel_size=7, activation="relu")(inputs)
pool_month = MaxPooling1D(pool_size=7)(conv_month)
#batch norm?
conv_week = Conv1D(filters=12, kernel_size=7, activation="relu")(pool_month)
pool_week = MaxPooling1D(pool_size=2)(conv_week)
conv_daily = Conv1D(7, 2, activation="relu")(pool_week)
pool_daily = MaxPooling1D(pool_size=2)(conv_daily)
flat = Flatten()(pool_daily)
outputs = Dense(28, activation="relu")(flat)

model = tf.keras.Model(inputs=inputs, outputs=outputs, name="what")
model.compile(optimizer='Adam', loss='mse')
model.summary()

NameError: ignored

In [0]:
model.fit(X_train, y_train, epochs=300,validation_data=(X_test, y_test))

Epoch 1/300
121/715 [====>.........................] - ETA: 25s - loss: 5.7379

KeyboardInterrupt: ignored

In [0]:
ha = X_test.iloc[0]
print(ha.shape)
ha = model.predict(ha[np.newaxis,:,np.newaxis])
he = y_test.iloc[0]
print(he.values)
print(ha)
print(tf.keras.losses.MSE(he,ha))

(1885,)
[0 0 1 1 0 0 0 0 1 0 1 1 1 3 2 0 0 0 4 1 1 0 1 0 0 0 0 0]
[[0.59340566 0.34249276 0.3614752  0.8593675  0.7911506  1.1059842
  1.2760688  0.93200886 0.86082894 0.5109823  0.6524267  0.7140881
  0.862646   1.3982621  0.7688996  0.63317597 0.63021034 0.2145137
  0.7491839  0.9524862  0.53070635 0.8116349  0.64785767 0.5203502
  0.54251087 0.6660379  0.7896682  0.34737346]]
tf.Tensor([0.8570064], shape=(1,), dtype=float32)


In [0]:
teateat